In [15]:
import pandas as pd
import numpy as np
import math as mt

In [16]:
event_training = pd.read_csv('training.csv')
event_test = pd.read_csv('test.csv')
training = pd.read_csv('labels_training_set.csv')
test = pd.read_csv('trocafone_kaggle_test.csv')

# Vamos a agregar algunos features relazionados con el tiempo entre eventos

# Primero vamos a calcular el promedio de tiempo entre eventos para cada usuario

In [17]:
# creo la columna date, la cual transforma el timestamp en una variable de tiempo
event_training['date'] =pd.to_datetime(event_training['timestamp'])

In [18]:
def crearColumnaTiempo(dataframe):
    person_cant_evento = dataframe.loc[:,['person','timestamp']].groupby('person').count()\
                                .reset_index()
    person_cant_evento.columns=['person','count_event']
    person_cant_evento_uno = person_cant_evento.loc[person_cant_evento['count_event']==1,:]
    person_cant_evento_mas_uno = person_cant_evento.loc[person_cant_evento['count_event']>1,:]
    person_cant_evento_uno.columns=['person','mean_time_between_events']
    person_cant_evento_uno['mean_time_between_events']=0
    tiempos = dataframe.loc[:,['person','date','Unnamed: 0','timestamp']]\
                    .merge(person_cant_evento_mas_uno,on='person',how='inner').sort_values(by='date',ascending=True)\
                         .groupby(['person','date','Unnamed: 0']).count().reset_index()\
                            .drop(['timestamp','count_event'],axis=1)
    tiempos['person_aux']=tiempos['person'].shift(-1,axis=0)
    tiempos['date_aux']=tiempos['date'].shift(-1,axis=0)
    tiempos['bool']=(tiempos['person']==tiempos['person_aux'])
    tiempos = tiempos.loc[tiempos['bool']==True,['person','date','date_aux']]
    tiempos['dis_time']=(tiempos['date_aux']-tiempos['date']).dt.total_seconds()
    tiempos = tiempos.loc[:,['person','dis_time']].groupby('person').mean().reset_index()
    tiempos.columns=['person','mean_time_between_events']
    event_time = tiempos.merge(person_cant_evento_uno,how='outer',on=['person','mean_time_between_events'])
    return event_time

In [19]:
event_time = crearColumnaTiempo(event_training)

/home/alfonso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Ahora vamos a calcular el promedio de tiempo entre eventos del mismo tipo para cada usuario, para eso defino la siguiente funcion:

In [20]:
def crearColumnaTiempoEvento(dataframe,evento,nombre_columna):  
    
    person_con = dataframe.groupby('person').filter(lambda x: (x['event']==evento).any())\
            .reset_index(drop=True)
    person_sin = dataframe.groupby('person').filter(lambda x: (x['event']!=evento).all())\
            .reset_index(drop=True)

    
    dis_time_sin = person_sin.loc[:,['person','event']].drop_duplicates(subset='person',keep='first')
    dis_time_sin.columns=['person',nombre_columna]
    dis_time_sin[nombre_columna]=-1

    cant = person_con.loc[person_con['event']==evento,['person','event']].groupby('person')\
                    .count().reset_index()
    cant.columns=['person','count']

    uno = cant.loc[cant['count']==1,:]
    uno.columns=['person',nombre_columna]
    uno[nombre_columna]=0
    uno = uno.drop_duplicates(keep='first')

    #ahora calculo el promedio de tiempo entre 'viewed product' por usuario.
    mas_uno = cant.loc[(cant['count']>1),:].merge(dataframe,on='person',how='inner')
    mas_uno = mas_uno.loc[mas_uno['event']==evento,['person','date','Unnamed: 0','timestamp']]\
                        .groupby(['person','date','Unnamed: 0'])\
                            .count().reset_index()\
                                .drop('timestamp',axis=1)

    mas_uno['date_aux']=mas_uno['date'].shift(-1,axis=0)
    mas_uno['person_aux']=mas_uno['person'].shift(-1,axis=0)
    mas_uno['bool']=(mas_uno['person_aux']==mas_uno['person'])

    mas_uno = mas_uno.loc[mas_uno['bool']==True,['person','date','date_aux']]

    mas_uno['time_between']=(mas_uno['date_aux']-mas_uno['date']).dt.total_seconds()
    mas_uno = mas_uno.loc[:,['person','time_between']].groupby('person').mean().reset_index()
    mas_uno.columns=['person',nombre_columna]

    event_time= pd.concat([mas_uno,uno,dis_time_sin])
    
    return event_time

In [21]:
#creo los dataframe
event_time_co = crearColumnaTiempoEvento(event_training,'conversion','mean_time_between_co')
event_time_ch = crearColumnaTiempoEvento(event_training,'checkout','mean_time_between_ch')
event_time_vs = crearColumnaTiempoEvento(event_training,'visited site','mean_time_between_vs')
event_time_ad = crearColumnaTiempoEvento(event_training,'ad campaign hit','mean_time_between_ad')
event_time_bl = crearColumnaTiempoEvento(event_training,'brand listing','mean_time_between_bl')
event_time_st = crearColumnaTiempoEvento(event_training,'staticpage','mean_time_between_st')
event_time_se = crearColumnaTiempoEvento(event_training,'search engine hit','mean_time_between_se')
event_time_sp = crearColumnaTiempoEvento(event_training,'searched products','mean_time_between_sp')
event_time_vp = crearColumnaTiempoEvento(event_training,'viewed product','mean_time_between_vp')
event_time_le = crearColumnaTiempoEvento(event_training,'lead','mean_time_between_le')
event_time_gl = crearColumnaTiempoEvento(event_training,'generic listing','mean_time_between_gl')

/home/alfonso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
#junto todos los dataframes en uno
train_time =  event_time.merge(event_time_co,on='person',how='inner')\
                        .merge(event_time_ch,on='person',how='inner')\
                        .merge(event_time_vs,on='person',how='inner')\
                        .merge(event_time_ad,on='person',how='inner')\
                        .merge(event_time_bl,on='person',how='inner')\
                        .merge(event_time_st,on='person',how='inner')\
                        .merge(event_time_se,on='person',how='inner')\
                        .merge(event_time_sp,on='person',how='inner')\
                        .merge(event_time_vp,on='person',how='inner')\
                        .merge(event_time_le,on='person',how='inner')\
                        .merge(event_time_gl,on='person',how='inner')

# paso la transformacion log a las columnas no numericas para normalizar los features

In [23]:
train_time.head()

,person,mean_time_between_events,mean_time_between_co,mean_time_between_ch,mean_time_between_vs,mean_time_between_ad,mean_time_between_bl,mean_time_between_st,mean_time_between_se,mean_time_between_sp,mean_time_between_vp,mean_time_between_le,mean_time_between_gl
0,0008ed71,2890.000000,-1.0,7225.0,9415.000000,-1.000000,-1.000000,-1.0,-1.00,-1.000000,-1.000000,-1.0,0.000000
1,000c79fe,38.750000,-1.0,0.0,0.000000,0.000000,-1.000000,-1.0,0.00,75.375000,214.500000,-1.0,0.000000
2,001802e4,35.888889,-1.0,0.0,0.000000,161.250000,-1.000000,-1.0,-1.00,21.333333,184.666667,-1.0,78.000000
3,0019e639,21016.589362,8838590.0,631371.0,548744.277778,352764.178571,60092.560976,-1.0,821093.75,881409.000000,52541.218085,-1.0,365842.703704
4,001b0bf9,153.000000,0.0,262.0,-1.000000,-1.000000,0.000000,-1.0,-1.00,-1.000000,10.000000,-1.0,0.000000


In [24]:
train_time['mean_time_between_co'] = train_time['mean_time_between_co']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time['mean_time_between_ch'] = train_time['mean_time_between_ch']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time['mean_time_between_ad'] = train_time['mean_time_between_ad']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time['mean_time_between_le'] = train_time['mean_time_between_le']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time['mean_time_between_vs'] = train_time['mean_time_between_vs']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time['mean_time_between_vp'] = train_time['mean_time_between_vp']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time['mean_time_between_events'] = train_time['mean_time_between_events']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time['mean_time_between_st'] = train_time['mean_time_between_st']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time['mean_time_between_gl'] = train_time['mean_time_between_gl']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time['mean_time_between_bl'] = train_time['mean_time_between_bl']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time['mean_time_between_se'] = train_time['mean_time_between_se']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time['mean_time_between_sp'] = train_time['mean_time_between_sp']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )

In [25]:
train_time.head()

,person,mean_time_between_events,mean_time_between_co,mean_time_between_ch,mean_time_between_vs,mean_time_between_ad,mean_time_between_bl,mean_time_between_st,mean_time_between_se,mean_time_between_sp,mean_time_between_vp,mean_time_between_le,mean_time_between_gl
0,0008ed71,3.460898,-1.000000,3.858838,3.97382,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.0,0.000000
1,000c79fe,1.588272,-1.000000,0.000000,0.00000,0.000000,-1.000000,-1.0,0.000000,1.877227,2.331427,-1.0,0.000000
2,001802e4,1.554960,-1.000000,0.000000,0.00000,2.207500,-1.000000,-1.0,-1.000000,1.329059,2.266389,-1.0,1.892095
3,0019e639,4.322562,6.946383,5.800285,5.73937,5.547484,4.778821,-1.0,5.914393,5.945177,4.720500,-1.0,5.563294
4,001b0bf9,2.184691,0.000000,2.418301,-1.00000,-1.000000,0.000000,-1.0,-1.000000,-1.000000,1.000000,-1.0,0.000000


# REPITO PARA EL SET DE PREDICCION

In [26]:
event_test['date'] =pd.to_datetime(event_test['timestamp'])

In [27]:
event_time_test = crearColumnaTiempo(event_test)

/home/alfonso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
event_time_co_test = crearColumnaTiempoEvento(event_test,'conversion','mean_time_between_co')
event_time_ch_test = crearColumnaTiempoEvento(event_test,'checkout','mean_time_between_ch')
event_time_vs_test = crearColumnaTiempoEvento(event_test,'visited site','mean_time_between_vs')
event_time_ad_test = crearColumnaTiempoEvento(event_test,'ad campaign hit','mean_time_between_ad')
event_time_bl_test = crearColumnaTiempoEvento(event_test,'brand listing','mean_time_between_bl')
event_time_st_test = crearColumnaTiempoEvento(event_test,'staticpage','mean_time_between_st')
event_time_se_test = crearColumnaTiempoEvento(event_test,'search engine hit','mean_time_between_se')
event_time_sp_test = crearColumnaTiempoEvento(event_test,'searched products','mean_time_between_sp')
event_time_vp_test = crearColumnaTiempoEvento(event_test,'viewed product','mean_time_between_vp')
event_time_le_test = crearColumnaTiempoEvento(event_test,'lead','mean_time_between_le')
event_time_gl_test = crearColumnaTiempoEvento(event_test,'generic listing','mean_time_between_gl')

/home/alfonso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
train_time_test =  event_time_test.merge(event_time_co_test,on='person',how='inner')\
                        .merge(event_time_ch_test,on='person',how='inner')\
                        .merge(event_time_vs_test,on='person',how='inner')\
                        .merge(event_time_ad_test,on='person',how='inner')\
                        .merge(event_time_bl_test,on='person',how='inner')\
                        .merge(event_time_st_test,on='person',how='inner')\
                        .merge(event_time_se_test,on='person',how='inner')\
                        .merge(event_time_sp_test,on='person',how='inner')\
                        .merge(event_time_vp_test,on='person',how='inner')\
                        .merge(event_time_le_test,on='person',how='inner')\
                        .merge(event_time_gl_test,on='person',how='inner')

In [ ]:
train_time_test.head()

In [ ]:
train_time_test['mean_time_between_co'] = train_time_test['mean_time_between_co']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time_test['mean_time_between_ch'] = train_time_test['mean_time_between_ch']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time_test['mean_time_between_ad'] = train_time_test['mean_time_between_ad']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time_test['mean_time_between_le'] = train_time_test['mean_time_between_le']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time_test['mean_time_between_vs'] = train_time_test['mean_time_between_vs']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time_test['mean_time_between_vp'] = train_time_test['mean_time_between_vp']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time_test['mean_time_between_events'] = train_time_test['mean_time_between_events']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time_test['mean_time_between_st'] = train_time_test['mean_time_between_st']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time_test['mean_time_between_gl'] = train_time_test['mean_time_between_gl']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time_test['mean_time_between_bl'] = train_time_test['mean_time_between_bl']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time_test['mean_time_between_se'] = train_time_test['mean_time_between_se']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )
train_time_test['mean_time_between_sp'] = train_time_test['mean_time_between_sp']\
                    .transform(lambda x: mt.log10(x) if (x > 0.0 ) else x )

In [ ]:
# defino los dataframe que uso para entrenar y predecir
features_train = pd.read_csv('train_paso_4.csv')
features_test = pd.read_csv('test_paso_4.csv')

In [ ]:
feature_train = training.merge(train_time,on='person',how='inner')\
                .merge(features_train,on=['person','label'],how='inner')
feature_test = test.merge(train_time_test,on='person',how='inner').merge(features_test,on='person',how='inner')

In [ ]:
feature_train.head()

In [ ]:
feature_test.head()

In [ ]:
feature_test.to_csv('test_paso_5.csv',index=False)
feature_train.to_csv('train_paso_5.csv',index=False)